In [8]:
from ipyleaflet import Map, Polygon, basemaps, LegendControl
from lxml import etree

import tqdm

In [24]:
# 用途地域の色を指定（マップに塗りつぶす色）
color_dic = {
    "第一種低層住居専用地域": "#440154",
    "第二種低層住居専用地域": "#481f70", 
    "第一種中高層住居専用地域": "#443983", 
    "第二種中高層住居専用地域": "#3b528b", 
    "第一種住居地域": "#31688e", 
    "第二種住居地域": "#287c8e", 
    "準住居地域": "#21918c", 
    "近隣商業地域": "#20a486", 
    "商業地域": "#35b779", 
    "準工業地域": "#5ec962", 
    "工業地域": "#90d743", 
    "工業専用地域": "#c8e020", 
    "不明": "#fde725", 
}

# 表示する用途地域のxmlファイルpath
xml_path = ""

In [28]:
def add_polygon_to_map(m, coordinates, kda):
    '''
    マップにポリゴンを加える
    '''
    polygon = Polygon(
        locations = coordinates,
        color = 'transparent' ,
        fill_color = color_dic[kda],
        fill_opacity = 0.6
    )

    m.add_layer(polygon)

def create_map(xml_path, color_dic):

    tree = etree.parse(xml_path) # XMLファイルを読み込む  
    root = tree.getroot() # 一番上の階層の要素を取り出します
    
    # マップを描画
    m = Map(center=(35.6894, 139.6917), zoom=15, basemap=basemaps.CartoDB.Positron) # 初期表示位置は東京
    # マップのサイズを指定
    m.layout.width = '1000px'
    m.layout.height = '600px'

    # 判例の設定
    legend = LegendControl(color_dic, position="topright")
    legend.style = {'width': '400px', 'height': '200px', 'font-size': '10px'}
    m.add_control(legend)
    
    # 名前空間を辞書に格納
    nsmap = {
        'ksj': 'http://nlftp.mlit.go.jp/ksj/schemas/ksj-app',
        'gml': 'http://www.opengis.net/gml',
        'xlink': 'http://www.w3.org/1999/xlink'
    }

    # 各エリアのポリゴンの情報を取得
    for area in tqdm.tqdm(root.findall('.//ksj:DesignatedArea', namespaces=nsmap)):
        surface_ref = area.find('ksj:cda', namespaces=root.nsmap).attrib['{http://www.w3.org/1999/xlink}href'].lstrip('#')
        use_name = area.find('ksj:kda', namespaces=root.nsmap).text
        curve_ref = root.find('.//gml:Surface[@gml:id="{}"]/gml:patches/gml:PolygonPatch/gml:exterior/gml:Ring/gml:curveMember'.format(surface_ref), namespaces=root.nsmap).attrib['{http://www.w3.org/1999/xlink}href'].lstrip('#')
        coords = root.find('.//gml:Curve[@gml:id="{}"]/gml:segments/gml:LineStringSegment/gml:posList'.format(curve_ref), namespaces=root.nsmap).text.split()
        coords = [(float(coords[i]), float(coords[i+1])) for i in range(0, len(coords), 2)]
        
        add_polygon_to_map(m, coords, use_name)

    
    m

In [29]:
create_map(xml_path, color_dic)

100%|██████████| 3183/3183 [00:12<00:00, 250.12it/s]
